## NLP Project

### Staring with downloading requried libaries and uploading the dataset

In [ ]:
# Install required libraries
!pip install nltk

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import ast
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/NLP-Project/resume_data.csv'

In [ ]:
# Load the dataset
df = pd.read_csv(path)

y = df.iloc[:, -1].values   # Last column

# Display the first few rows
df.head()

,career_objective,skills,educational_institution_name,degree_names,passing_years,major_field_of_studies,professional_company_names,start_dates,end_dates,related_skils_in_job,positions,locations,responsibilities,certification_providers,certification_skills,job_position_name,job_category
0,NaN,"['R', 'Python', 'Tableau', 'Power BI', 'SQL', ...",['NARSEE MONJEE INSTITUTE'],['BCA'],['2020'],['N/A'],['DENA FUNDS'],['APR 2020'],['TILL DATE'],"[['predictive data analysis', 'large scale dat...",['BUSINESS ANALYST'],['N/A'],Administrative Support\nScheduling\nFiling & D...,['N/A'],[None],Asst. Manager/ Manger (Administrative),Business Management
1,NaN,"['streamline', 'approach', 'balance sheet', 'b...",['McCallum Graduate School at Bentley Universi...,"[""Master's in Finance"", 'Bachelor of Science']","['December 2012', 'May 2011']","['Finance', 'Economics & Finance']","['PainQx', 'LSN', 'Abpro']","['01/2016', '06/2012', '02/2011']","['Current', '01/2016', '07/2012']","[['business development', 'CRM', 'financials',...","['VP of Business Development', 'Co-Founder VP ...","['City, State', 'City, State', 'City, State']",Administrative Support\nScheduling\nFiling & D...,NaN,NaN,Asst. Manager/ Manger (Administrative),Business Management
2,"A curiosity-driven data scientist, eager to le...","['Data Analysis', 'Machine learning', 'Data Mi...","['MamCO University', 'MamCO University']","['BSc(CA)', 'MSc(CA)']","['2018', '2020']","['Computer Applications', 'Computer Applicatio...",['KPMMC Inc.'],['Aug 2020'],['Ongoing'],"[['Natural Language Processing', 'Python', 'Ma...",['Machine Learning Junior Developer'],['N/A'],Administrative Support\nScheduling\nFiling & D...,['N/A'],"[['Big Data Analysis', 'Hadoop']]",Asst. Manager/ Manger (Administrative),Business Management
3,To pursue excellence in the dynamic business w...,"['3d', 'accounting', 'Accountant', 'Accounts r...","['Vidyasagar University', 'Mohan Lal Sukhadia ...","['MBA', 'N/A']","['2006', '2000']","['Finance', 'N/A']","['Company Name', 'Company Name', 'Auto1 (Al-Gh...","['October 2011', 'January 2009', 'August 2007'...","['August 2015', 'September 2011', 'December 20...","[['accounting'], ['accounting'], ['Accounting'...","['Accountant', 'Accountant', 'Accountant', 'Ac...","['City, State', 'City, State', 'Kuwait', 'City...",Administrative Support\nScheduling\nFiling & D...,NaN,NaN,Asst. Manager/ Manger (Administrative),Business Management
4,NaN,"['QA Engineering software expertise - Design, ...","['Devi Ahilya University', 'Devi Ahilya Univer...","['Master of Computer Applications (MCA)', 'Bac...","['2005', '2002']","['Computer Applications, Software Engineering,...","['Company Name', 'Company Name', 'Company Name']","['01/2016', '04/2015', '09/2005']","['Current', '01/2016', '04/2015']","[None, None, None]","['QA Engineering Manager', 'Software Certifica...","['City , State', 'City , State', 'City , State']",Administrative Support\nScheduling\nFiling & D...,"['Certified Scrum Master (CSM)', 'Certified Te...","[None, None, None, ['Python'], ['Python: XML, ...",Asst. Manager/ Manger (Administrative),Business Management


### Cleaning the data so that all of the empty field are replaced with NaN

In [ ]:
# Function to fix lists: turn ['N/A'], ['None'] etc. → np.nan
def clean_list_field(cell):
    try:
        if isinstance(cell, str) and cell.startswith('['):
            parsed = ast.literal_eval(cell)
            if isinstance(parsed, list):
                cleaned = [item for item in parsed if str(item).strip().lower() not in ['n/a', 'none', 'nan']]
                return cleaned if cleaned else np.nan
        return cell
    except Exception:
        return np.nan

# Apply cleaning function to each relevant column
for col in df.columns:
      df[col] = df[col].apply(clean_list_field)

# Coverting all of the list into single string
for col in df.columns:
      df[col] = df[col].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else x)

# Display cleaned and flattened data
df.head()


,career_objective,skills,educational_institution_name,degree_names,passing_years,major_field_of_studies,professional_company_names,start_dates,end_dates,related_skils_in_job,positions,locations,responsibilities,certification_providers,certification_skills,job_position_name,job_category
0,NaN,R Python Tableau Power BI SQL SAS Deep Learnin...,NARSEE MONJEE INSTITUTE,BCA,2020,NaN,DENA FUNDS,APR 2020,TILL DATE,"['predictive data analysis', 'large scale data...",BUSINESS ANALYST,NaN,Administrative Support\nScheduling\nFiling & D...,NaN,NaN,Asst. Manager/ Manger (Administrative),Business Management
1,NaN,streamline approach balance sheet bonds budget...,McCallum Graduate School at Bentley University...,Master's in Finance Bachelor of Science,December 2012 May 2011,Finance Economics & Finance,PainQx LSN Abpro,01/2016 06/2012 02/2011,Current 01/2016 07/2012,"['business development', 'CRM', 'financials', ...",VP of Business Development Co-Founder VP of Bu...,"City, State City, State City, State",Administrative Support\nScheduling\nFiling & D...,NaN,NaN,Asst. Manager/ Manger (Administrative),Business Management
2,"A curiosity-driven data scientist, eager to le...",Data Analysis Machine learning Data Mining Nat...,MamCO University MamCO University,BSc(CA) MSc(CA),2018 2020,Computer Applications Computer Applications,KPMMC Inc.,Aug 2020,Ongoing,"['Natural Language Processing', 'Python', 'Mac...",Machine Learning Junior Developer,NaN,Administrative Support\nScheduling\nFiling & D...,NaN,"['Big Data Analysis', 'Hadoop']",Asst. Manager/ Manger (Administrative),Business Management
3,To pursue excellence in the dynamic business w...,3d accounting Accountant Accounts receivables ...,Vidyasagar University Mohan Lal Sukhadia Unive...,MBA,2006 2000,Finance,Company Name Company Name Auto1 (Al-Ghannam Au...,October 2011 January 2009 August 2007 April 20...,August 2015 September 2011 December 2008 May 2...,"['accounting'] ['accounting'] ['Accounting', '...",Accountant Accountant Accountant Accountant Ac...,"City, State City, State Kuwait City, State Cit...",Administrative Support\nScheduling\nFiling & D...,NaN,NaN,Asst. Manager/ Manger (Administrative),Business Management
4,NaN,"QA Engineering software expertise - Design, Es...",Devi Ahilya University Devi Ahilya University,Master of Computer Applications (MCA) Bachelor...,2005 2002,"Computer Applications, Software Engineering, D...",Company Name Company Name Company Name,01/2016 04/2015 09/2005,Current 01/2016 04/2015,NaN,QA Engineering Manager Software Certification ...,"City , State City , State City , State",Administrative Support\nScheduling\nFiling & D...,Certified Scrum Master (CSM) Certified Tester ...,"['Python'] ['Python: XML, JSON, and the Web'] ...",Asst. Manager/ Manger (Administrative),Business Management


### 1. Preparing the Dataset for Job Category Prediction
In this section, we are preparing our structured resume dataset for a classification task where the goal is to predict the job category that best fits each candidate based on their resume content.

Our dataset already contains extracted fields like skills, degrees, previous positions, and certifications. Instead of working with raw resume text, we are using these structured fields as inputs to our model.

Here’s what we’re doing:

1. Select key resume sections (features) that are most relevant for predicting job roles (e.g., skills, degree_names, positions, etc.). We already picked out the features and updated our dataset with them.

2. Use CountVectorizer() separately for each text-based column to transform the selected resume fields into numerical feature vectors. This will treat each column as a bag-of-words and converts it into a matrix of token counts. After vectorizing each feature column, we concatenate all the resulting matrices horizontally into a single input feature matrix for modeling.

3. Encode the job categories into numeric labels using LabelEncoder.

4. Split the dataset into training and testing sets, so we can evaluate our model’s performance later.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 1. Select all columns except the last two
X_df = df.drop(columns=["job_category", "job_position_name"], errors="ignore").copy()

# Convert all feature columns to string (for CountVectorizer)
X_df = X_df.astype(str)

# 2. Initialize vectorizers and transformed arrays
vectorizers = {}
vectors = []

# Apply CountVectorizer to each feature column
for col in X_df.columns:
    # Ensure column isn't completely empty or just whitespace
    if X_df[col].str.strip().eq('').all():
        print(f"Skipping column '{col}' — all entries are empty after cleaning.")
        continue

    try:
        vec = CountVectorizer()
        X_col = vec.fit_transform(X_df[col])
        if not vec.vocabulary_:
            print(f"Skipping column '{col}' — vocabulary is empty.")
            continue
        vectorizers[col] = vec
        vectors.append(X_col.toarray())
    except ValueError as e:
        print(f"Skipping column '{col}' due to ValueError: {e}")


# Concatenate all vectors horizontally
X = np.concatenate(vectors, axis=1)

# 3. Encode the target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['job_category'])

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_df.head()

,career_objective,skills,educational_institution_name,degree_names,passing_years,major_field_of_studies,professional_company_names,start_dates,end_dates,related_skils_in_job,positions,locations,responsibilities,certification_providers,certification_skills
0,nan,R Python Tableau Power BI SQL SAS Deep Learnin...,NARSEE MONJEE INSTITUTE,BCA,2020,nan,DENA FUNDS,APR 2020,TILL DATE,"['predictive data analysis', 'large scale data...",BUSINESS ANALYST,nan,Administrative Support\nScheduling\nFiling & D...,nan,nan
1,nan,streamline approach balance sheet bonds budget...,McCallum Graduate School at Bentley University...,Master's in Finance Bachelor of Science,December 2012 May 2011,Finance Economics & Finance,PainQx LSN Abpro,01/2016 06/2012 02/2011,Current 01/2016 07/2012,"['business development', 'CRM', 'financials', ...",VP of Business Development Co-Founder VP of Bu...,"City, State City, State City, State",Administrative Support\nScheduling\nFiling & D...,nan,nan
2,"A curiosity-driven data scientist, eager to le...",Data Analysis Machine learning Data Mining Nat...,MamCO University MamCO University,BSc(CA) MSc(CA),2018 2020,Computer Applications Computer Applications,KPMMC Inc.,Aug 2020,Ongoing,"['Natural Language Processing', 'Python', 'Mac...",Machine Learning Junior Developer,nan,Administrative Support\nScheduling\nFiling & D...,nan,"['Big Data Analysis', 'Hadoop']"
3,To pursue excellence in the dynamic business w...,3d accounting Accountant Accounts receivables ...,Vidyasagar University Mohan Lal Sukhadia Unive...,MBA,2006 2000,Finance,Company Name Company Name Auto1 (Al-Ghannam Au...,October 2011 January 2009 August 2007 April 20...,August 2015 September 2011 December 2008 May 2...,"['accounting'] ['accounting'] ['Accounting', '...",Accountant Accountant Accountant Accountant Ac...,"City, State City, State Kuwait City, State Cit...",Administrative Support\nScheduling\nFiling & D...,nan,nan
4,nan,"QA Engineering software expertise - Design, Es...",Devi Ahilya University Devi Ahilya University,Master of Computer Applications (MCA) Bachelor...,2005 2002,"Computer Applications, Software Engineering, D...",Company Name Company Name Company Name,01/2016 04/2015 09/2005,Current 01/2016 04/2015,nan,QA Engineering Manager Software Certification ...,"City , State City , State City , State",Administrative Support\nScheduling\nFiling & D...,Certified Scrum Master (CSM) Certified Tester ...,"['Python'] ['Python: XML, JSON, and the Web'] ..."


In [ ]:
# Sum token counts across all feature columns
tokens_per_sample = np.sum(X, axis=1)

# Summary stats
print("Average tokens per sample:", np.mean(tokens_per_sample))
print("Min tokens per sample:", np.min(tokens_per_sample))
print("Max tokens per sample:", np.max(tokens_per_sample))


Average tokens per sample: 139.60689402273562
Min tokens per sample: 37
Max tokens per sample: 526


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assume this is already defined
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['job_category'])

# Print category → encoded value mapping
for i, label in enumerate(label_encoder.classes_):
    print(f"{label}: {i}")


Business Management: 0
Civil and Mechanical Engineering: 1
HR & Marketing: 2
Software & IT: 3
Tech Engineering: 4


## 2. Baseline Classification
We will evaluate three simple strategies using 5-fold cross-validation:

- **most_frequent**: always predict the most common class
- **stratified**: predict randomly according to class distribution
- **uniform**: predict randomly with equal chance for each class


In [ ]:
# Count how many samples exist for each job category
category_counts = df['job_category'].value_counts()

# Display the counts
print("Number of samples per job category:\n")
print(category_counts)


Number of samples per job category:

job_category
Software & IT                       2384
Civil and Mechanical Engineering    1708
Tech Engineering                    1701
Business Management                 1364
HR & Marketing                      1024
Name: count, dtype: int64


In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Create scorers for multiclass
scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro', zero_division=0),
    'recall': make_scorer(recall_score, average='macro', zero_division=0),
    'f1': make_scorer(f1_score, average='macro', zero_division=0)
}

# Run dummy classifiers
for strategy in ["most_frequent", "stratified", "uniform"]:
    print(f"\n--- Strategy: {strategy} ---")
    dummy = DummyClassifier(strategy=strategy, random_state=42)
    scores = cross_validate(dummy, X, y, cv=5, scoring=scoring_metrics)

    # Print average results
    for metric in scoring_metrics:
        print(f"{metric}: {scores['test_' + metric].mean():.4f}")



--- Strategy: most_frequent ---
accuracy: 0.2914
precision: 0.0583
recall: 0.2000
f1: 0.0903

--- Strategy: stratified ---
accuracy: 0.2233
precision: 0.2088
recall: 0.2087
f1: 0.2086

--- Strategy: uniform ---
accuracy: 0.2056
precision: 0.2058
recall: 0.2081
f1: 0.2029


## 3. Naive Bayes Classification

Here, we decided to use Multinomial Naive Bayes because it works well with discrete data, such as word counts or term frequencies. We continue to use 5-fold cross validation, but here we will print out the results separately for each fold.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score


# Initialize the classifier
mnb = MultinomialNB()

# Run 5-fold cross-validation
nb_scores = cross_validate(mnb, X, y, cv=5, scoring=scoring_metrics, return_train_score=False)

# Print individual fold results
print("Multinomial Naive Bayes (Bag-of-Words) Performance (each fold):")
for metric in scoring_metrics.keys():
    print(f"{metric}: {nb_scores['test_' + metric]}")

# Print average scores
print("\nMultinomial Naive Bayes (Bag-of-Words) Average Performance:")
for metric in scoring_metrics.keys():
    mean_score = nb_scores['test_' + metric].mean()
    print(f"{metric}: {mean_score:.4f}")


Multinomial Naive Bayes (Bag-of-Words) Performance (each fold):
accuracy: [0.78497251 0.7891198  0.79339853 0.92787286 0.99694377]
precision: [0.86649498 0.87718919 0.89354819 0.95848628 0.9952381 ]
recall: [0.85038352 0.7981112  0.85748535 0.93061282 0.99757255]
f1: [0.79072597 0.71877768 0.82168141 0.93550547 0.99637294]

Multinomial Naive Bayes (Bag-of-Words) Average Performance:
accuracy: 0.8585
precision: 0.9182
recall: 0.8868
f1: 0.8526


## 4. Statistical Classification: Logistic Regression and Linear SVC

In this step, we will evaluate two common statistical models — Logistic Regression and Linear Support Vector Classifier (SVC) — to predict a candidate's job category using structured resume data. We continue to use 5-fold cross validation.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# --- Logistic Regression ---
print("\n--- Logistic Regression ---")
log_reg = LogisticRegression(max_iter=1000)
log_scores = cross_validate(log_reg, X, y, cv=5, scoring=scoring_metrics)

for metric in scoring_metrics:
    print(f"{metric}: {log_scores['test_' + metric].mean():.4f}")

# --- Linear SVC ---
print("\n--- Linear SVC ---")
svc = LinearSVC(max_iter=10000)
svc_scores = cross_validate(svc, X, y, cv=5, scoring=scoring_metrics)

for metric in scoring_metrics:
    print(f"{metric}: {svc_scores['test_' + metric].mean():.4f}")



--- Logistic Regression ---
accuracy: 0.8269
precision: 0.8739
recall: 0.8567
f1: 0.8165

--- Linear SVC ---
accuracy: 0.7918
precision: 0.8589
recall: 0.8231
f1: 0.7681


## 5. Neural Classification with Feed Forward Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# --- FFNN ---
print("\n--- FFNN ---")
ffnn = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(64, 32, 16, 8, 4), random_state=1)
ffnn.fit(X_train, y_train)

ffnn_scores = cross_validate(ffnn, X, y, cv=5, scoring=scoring_metrics)

for metric in scoring_metrics:
    print(f"{metric}: {ffnn_scores['test_' + metric].mean():.4f}")


--- FFNN ---
accuracy: 0.7436
precision: 0.7913
recall: 0.7766
f1: 0.7324


## 6. Statistical Approaches with Word2Vec

To improve the semantic understanding of text in resume fields (like Skills, Education, Certifications), we will use pre trained Word2Vec embeddings. These embeddings help capture the meaning and similarity between words — for example, "Python" and "Java" will be closer in the vector space than unrelated terms. Instead of sparse word counts, this gives us rich numerical representations.

In [ ]:
!wget https://github.com/eyaler/word2vec-slim/raw/refs/heads/master/GoogleNews-vectors-negative300-SLIM.bin.gz

--2025-04-18 22:17:37--  https://github.com/eyaler/word2vec-slim/raw/refs/heads/master/GoogleNews-vectors-negative300-SLIM.bin.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/eyaler/word2vec-slim/refs/heads/master/GoogleNews-vectors-negative300-SLIM.bin.gz [following]
--2025-04-18 22:17:37--  https://media.githubusercontent.com/media/eyaler/word2vec-slim/refs/heads/master/GoogleNews-vectors-negative300-SLIM.bin.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276467217 (264M) [application/octet-stream]
Saving to: ‘GoogleNews-vectors-negative300-SLIM.bin.gz’

GoogleNews-vecto

In [ ]:
!pip install gensim
!pip install numpy==1.24.3 --force-reinstall
!pip install scipy

In [ ]:
# Step 3: Import libraries again
import gensim
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler

# Load the Word2Vec model
bigmodel = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300-SLIM.bin.gz", binary=True)

# Function to get average Word2Vec vector
def get_avg_w2v(text):
    tokens = str(text).split()
    vectors = [bigmodel[word] for word in tokens if word in bigmodel]
    if len(vectors) == 0:
        return np.zeros(300)
    return np.mean(vectors, axis=0)

# Stack all vectors and scale
X_w2v_combined = np.hstack(vectors)
X_w2v_combined = StandardScaler().fit_transform(X_w2v_combined)

# Train a Logistic Regression model with 5-fold CV
clf = LogisticRegression(max_iter=1000)
scores = cross_validate(clf, X_w2v_combined, y, cv=5, scoring=scoring_metrics)

# Display Results
print("--- Logistic Regression (Word2Vec with Cross-Validation) ---")
for metric in scoring_metrics:
    print(f"{metric}: {scores['test_' + metric].mean():.4f}")

--- Logistic Regression (Word2Vec with Cross-Validation) ---
accuracy: 0.8126
precision: 0.8890
recall: 0.8544
f1: 0.8108


## 7. Neural Approaches with Word2Vec (MLP Classifier)

## 8. Using BERT Embedding for Classification

In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install numpy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

# STEP 1: Combine text features into a single "text" column
X_df = df.drop(columns=["job_position_name", "job_category", "label"], errors='ignore').copy()
df['text'] = X_df.apply(
    lambda row: ' '.join(
        str(val).replace('\n', ', ').strip()
        for val in row
        if pd.notna(val) and str(val).strip().lower() != 'nan'
    ),
    axis=1
)
df['text'] = df['text'].str.strip().str.replace(r'\s+', ' ', regex=True)

# STEP 2: Encode labels
df['label'] = pd.Categorical(df['job_category']).codes

# STEP 3: Use sklearn to split into train/test
df_train, df_test = train_test_split(df[['text', 'label']], test_size=0.2, random_state=34, stratify=df['label'])

# STEP 4: Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)



In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)


Map:   0%|          | 0/6544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1637 [00:00<?, ? examples/s]

In [ ]:
print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")

for i in range(5):
    print("LABEL:", df['job_category'].iloc[i])
    print("TEXT:", df['text'].iloc[i])
    print("----")

# Check original label distribution
print(df['label'].value_counts())

# Convert to pandas for label distribution check
train_df = train_dataset.to_pandas()
test_df = test_dataset.to_pandas()

# Check class distribution
print("Train label distribution:")
print(train_df['label'].value_counts())

print("\nTest label distribution:")
print(test_df['label'].value_counts())


Training set size: 6544
Test set size: 1637
LABEL: Business Management
TEXT: R Python Tableau Power BI SQL SAS Deep Learning Neural Networks Artificial Intelligence NARSEE MONJEE INSTITUTE BCA 2020 DENA FUNDS APR 2020 TILL DATE ['predictive data analysis', 'large scale data handling'] BUSINESS ANALYST Administrative Support, Scheduling, Filing & Documentation, Communication, Team Support, Equipment Maintenance, Information Provision, Inventory Management, Team Collaboration, OHS Policy Development, Safety Advice, Risk Assessment, Policy Review, OHS Training, Safety Inspections, Unsafe Act Prevention, Incident Investigation, Report Preparation Asst. Manager/ Manger (Administrative) Business Management
----
LABEL: Business Management
TEXT: streamline approach balance sheet bonds budgeting business development cash flow clinical research clinical trials com consulting corporate finance CRM database email English equities Fast financials financing financial fundraising funds graphical user

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = len(df['label'].unique())  # Or set(df['label'])
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorWithPadding

# Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from sklearn.metrics import classification_report
from evaluate import load

# Metrics
def compute_metrics(eval_pred):
   load_accuracy = load("accuracy")
   load_f1 = load("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./bert_resume_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

<ipython-input-24-17ccadc0671d>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
500,0.221600
1000,0.021800


TrainOutput(global_step=1227, training_loss=0.10310326772874803, metrics={'train_runtime': 807.8848, 'train_samples_per_second': 24.3, 'train_steps_per_second': 1.519, 'total_flos': 2269701923836800.0, 'train_loss': 0.10310326772874803, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.021392472088336945,
 'eval_accuracy': 0.9908368967623702,
 'eval_f1': 0.9918573652250698,
 'eval_runtime': 22.4817,
 'eval_samples_per_second': 72.815,
 'eval_steps_per_second': 4.582,
 'epoch': 3.0}

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the tokenized test set
pred = trainer.predict(tokenized_test)
y_pred = np.argmax(pred.predictions, axis=-1)
y_true = pred.label_ids

# Generate classification report
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.9890    0.9945       273
           1     1.0000    0.9912    0.9956       342
           2     1.0000    0.9902    0.9951       205
           3     0.9695    1.0000    0.9845       477
           4     1.0000    0.9794    0.9896       340

    accuracy                         0.9908      1637
   macro avg     0.9939    0.9900    0.9919      1637
weighted avg     0.9911    0.9908    0.9909      1637



The result above shows an accuracy of 100% and a recall rate of 100%, which is really good, but suspiciously good. To make sure that the model is not just memorizing the results, we train our model with 5 fold cross validation using DistilBERT.

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
import os

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

# STEP 1: Combine features into a single text column
X_df = df.drop(columns=["job_position_name", "job_category"])

# Combine all features
df['text'] = X_df.apply(
    lambda row: ' '.join(
        str(val).replace('\n', ', ').strip()
        for val in row
        if pd.notna(val) and str(val).strip().lower() != 'nan'
    ),
    axis=1
)
df['text'] = df['text'].str.strip().str.replace(r'\s+', ' ', regex=True)

# STEP 2: Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['job_category'])

# STEP 3: Create unseen test set (20%)
df_train_full, df_unseen = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# STEP 4: Split remaining into train (90%) and validation (10%)
df_train, df_val = train_test_split(df_train_full, test_size=0.1, stratify=df_train_full['label'], random_state=42)

# STEP 5: Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
unseen_dataset = Dataset.from_pandas(df_unseen)

# STEP 6: Tokenizer and Preprocessing
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_dataset.map(preprocess_function)
tokenized_val = val_dataset.map(preprocess_function)
tokenized_unseen = unseen_dataset.map(preprocess_function)

# STEP 7: Model Setup
num_labels = len(df['label'].unique())
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# STEP 8: Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro", zero_division=0)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# STEP 9: Training Arguments
training_args = TrainingArguments(
    output_dir="./bert_resume_final_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="no",
    report_to="none"
)

# STEP 10: Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate on unseen test set
pred = trainer.predict(tokenized_unseen)
y_pred = np.argmax(pred.predictions, axis=-1)
y_true = pred.label_ids

print("\nUnseen Test Set Classification Report:")
print(classification_report(y_true, y_pred, digits=4))


Map:   0%|          | 0/5889 [00:00<?, ? examples/s]

Map:   0%|          | 0/655 [00:00<?, ? examples/s]

Map:   0%|          | 0/1637 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-30-a2c05d0eceef>:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


In [ ]:
X_df.head()


,career_objective,skills,educational_institution_name,degree_names,passing_years,major_field_of_studies,professional_company_names,start_dates,end_dates,related_skils_in_job,positions,locations,responsibilities,certification_providers,certification_skills,job_position_name,job_category
0,NaN,R Python Tableau Power BI SQL SAS Deep Learnin...,NARSEE MONJEE INSTITUTE,BCA,2020,NaN,DENA FUNDS,APR 2020,TILL DATE,"['predictive data analysis', 'large scale data...",BUSINESS ANALYST,NaN,Administrative Support\nScheduling\nFiling & D...,NaN,NaN,Asst. Manager/ Manger (Administrative),Business Management
1,NaN,streamline approach balance sheet bonds budget...,McCallum Graduate School at Bentley University...,Master's in Finance Bachelor of Science,December 2012 May 2011,Finance Economics & Finance,PainQx LSN Abpro,01/2016 06/2012 02/2011,Current 01/2016 07/2012,"['business development', 'CRM', 'financials', ...",VP of Business Development Co-Founder VP of Bu...,"City, State City, State City, State",Administrative Support\nScheduling\nFiling & D...,NaN,NaN,Asst. Manager/ Manger (Administrative),Business Management
2,"A curiosity-driven data scientist, eager to le...",Data Analysis Machine learning Data Mining Nat...,MamCO University MamCO University,BSc(CA) MSc(CA),2018 2020,Computer Applications Computer Applications,KPMMC Inc.,Aug 2020,Ongoing,"['Natural Language Processing', 'Python', 'Mac...",Machine Learning Junior Developer,NaN,Administrative Support\nScheduling\nFiling & D...,NaN,"['Big Data Analysis', 'Hadoop']",Asst. Manager/ Manger (Administrative),Business Management
3,To pursue excellence in the dynamic business w...,3d accounting Accountant Accounts receivables ...,Vidyasagar University Mohan Lal Sukhadia Unive...,MBA,2006 2000,Finance,Company Name Company Name Auto1 (Al-Ghannam Au...,October 2011 January 2009 August 2007 April 20...,August 2015 September 2011 December 2008 May 2...,"['accounting'] ['accounting'] ['Accounting', '...",Accountant Accountant Accountant Accountant Ac...,"City, State City, State Kuwait City, State Cit...",Administrative Support\nScheduling\nFiling & D...,NaN,NaN,Asst. Manager/ Manger (Administrative),Business Management
4,NaN,"QA Engineering software expertise - Design, Es...",Devi Ahilya University Devi Ahilya University,Master of Computer Applications (MCA) Bachelor...,2005 2002,"Computer Applications, Software Engineering, D...",Company Name Company Name Company Name,01/2016 04/2015 09/2005,Current 01/2016 04/2015,NaN,QA Engineering Manager Software Certification ...,"City , State City , State City , State",Administrative Support\nScheduling\nFiling & D...,Certified Scrum Master (CSM) Certified Tester ...,"['Python'] ['Python: XML, JSON, and the Web'] ...",Asst. Manager/ Manger (Administrative),Business Management


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.preprocessing import LabelEncoder
from evaluate import load
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
import os

os.environ["WANDB_DISABLED"] = "true"

# STEP 1: Combine text features into a single "text" column
X_df = df.iloc[:, :-2]  # exclude last two columns if those are not features
df['text'] = X_df.apply(
    lambda row: ' '.join(
        str(val).replace('\n', ', ').strip()
        for val in row
        if pd.notna(val) and str(val).strip().lower() != 'nan'
    ),
    axis=1
)
df['text'] = df['text'].str.strip().str.replace(r'\s+', ' ', regex=True)

# STEP 2: Encode labels
df['label'] = pd.Categorical(df['job_category']).codes

# STEP 3: Use sklearn to split into train/test
df_train, df_test = train_test_split(df[['text', 'label']], test_size=0.2, random_state=42, stratify=df['label'])

# STEP 4: Sample 3000 training and 300 testing examples
train_df_sampled = df_train.sample(n=3000, random_state=42).reset_index(drop=True)
test_df_sampled = df_test.sample(n=300, random_state=42).reset_index(drop=True)

# Prepare text and label arrays for CV
texts = np.array(train_df_sampled["text"].tolist())
labels = np.array(train_df_sampled["label"].tolist())

# Tokenizer and collator
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro", zero_division=0)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
f1_scores = []
num_labels = len(np.unique(labels))

for fold, (train_idx, val_idx) in enumerate(kf.split(texts, labels)):
    print(f"\nFold {fold + 1}")

    train_texts = texts[train_idx]
    train_labels = labels[train_idx]
    val_texts = texts[val_idx]
    val_labels = labels[val_idx]

    train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()})
    val_dataset = Dataset.from_dict({"text": val_texts.tolist(), "label": val_labels.tolist()})

    tokenized_train = train_dataset.map(preprocess_function)
    tokenized_val = val_dataset.map(preprocess_function)

    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

    training_args = TrainingArguments(
        output_dir=f"./results_fold_{fold+1}",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        logging_dir="./logs",
        report_to="none",
        save_strategy="no"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluate and print macro metrics
    eval_result = trainer.evaluate()
    print(f"\nFold {fold + 1} Summary Metrics:")
    for metric, value in eval_result.items():
        print(f"  {metric}: {value:.4f}")

    # Predict and print per-class classification report
    predictions = trainer.predict(tokenized_val)
    y_pred = np.argmax(predictions.predictions, axis=-1)
    y_true = predictions.label_ids

    print(f"\nFold {fold + 1} Classification Report:")
    print(classification_report(y_true, y_pred, digits=4))

    accuracies.append(eval_result["eval_accuracy"])
    f1_scores.append(eval_result["eval_f1"])

# Final cross-validation summary
print("\nCross-validation results:")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-98a21d044d6d>:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.255200



Fold 5 Summary Metrics:
  eval_loss: 0.0193
  eval_accuracy: 0.9917
  eval_precision: 0.9924
  eval_recall: 0.9917
  eval_f1: 0.9920
  eval_runtime: 7.6414
  eval_samples_per_second: 78.5190
  eval_steps_per_second: 4.9730
  epoch: 5.0000

Fold 5 Classification Report:
              precision    recall  f1-score   support

           0     1.0000    0.9804    0.9901       102
           1     0.9843    0.9921    0.9881       126
           2     1.0000    1.0000    1.0000        76
           3     0.9943    0.9943    0.9943       174
           4     0.9837    0.9918    0.9878       122

    accuracy                         0.9917       600
   macro avg     0.9924    0.9917    0.9920       600
weighted avg     0.9917    0.9917    0.9917       600


Cross-validation results:
Average Accuracy: 0.9887
Average F1 Score: 0.9884


## 9. LLMs for Few-Shot Learning or Embedding-Based Classification

Here, we will be using Mistral's LLM with both zero-shot and few-shot prompting

### Installing and Setting up Mistral API

In [ ]:
pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.5/301.5 kB 7.5 MB/s eta 0:00:00


In [ ]:
from mistralai import Mistral

# Here is where you need to tell your code your mistral API key.
# If you skipped the intructions above, create an account and get one here:
# https://console.mistral.ai/home

client = Mistral(api_key="w6yh0tpXA6gK3p52Yt1vfzM8Yv74LkNX")

In [ ]:
# This is the large latest LLM from Mistral.
# There are other models. You can find out about them here;
# https://docs.mistral.ai/getting-started/models/models_overview/
MODEL = "mistral-large-latest"

In [ ]:
# Combine all text features into one "text" column
feature_columns = df.columns[df.columns != 'job_category']
df['text'] = df[feature_columns].apply(
    lambda row: ' '.join(
        str(val).replace('\n', ' ').strip()
        for val in row
        if pd.notna(val) and str(val).strip().lower() != 'nan'
    ),
    axis=1
)

# Encode labels
df['label'] = pd.Categorical(df['job_category']).codes

In [ ]:
# Sample 500 resumes
resume_sampled_df = df.sample(n=500, random_state=42).reset_index(drop=True)

### Zero Shot Prompting
In zero-shot prompting, we give the model only a basic instruction without any examples.

We prompt the model to classify each resume into one of the predefined job categories. This tests the model's generalization abilities based on your instructions alone.



In [ ]:
import time
import pandas as pd

# Store predictions
predictions = []

# Loop through the sampled resumes
for index, row in resume_sampled_df.iterrows():
    # Create the prompt using the text field from the current row
    prompt = (
        "Based on the following resume details, predict the job category from the following options:\n"
        "0: Business Management\n"
        "1: Civil and Mechanical Engineering\n"
        "2: HR & Marketing\n"
        "3: Software & IT\n"
        "4: Tech Engineering\n"
        f"Resume:\n{row['text']}\n"
        "Answer with only the integer corresponding to the category."
    )

    # Put it in MESSAGES format for Mistral
    MESSAGES = [{"role": "user", "content": prompt}]


    # Make the LLM request
    completion = client.chat.complete(
        model=MODEL,
        messages=MESSAGES
    )
    response = completion.choices[0].message.content.strip()

    # Print prompt and response
    print(f"Prompt:\n{prompt}")
    print(f"Prediction:\n{completion.choices[0].message.content}")

    # Save the response
    match = re.search(r"\b([0-4])\b", response)
    predictions.append(int(match.group(1)) if match else "ERROR")

    # Pause to avoid hitting Mistral rate limits
    time.sleep(5)


Streaming output truncated to the last 5000 lines.
Resume:
To obtain a position in a fast-paced business office environment, demanding a strong organizational, technical, and interpersonal position utilizing my skills and attributes. accounts payables accounts receivables Accounts Payable Accounts Receivable administrative functions trial balance banking budget bi closing Computer Applications Credit clients Customer Service data entry delivery driving email insurance inventory ledger Access Excel Outlook PowerPoint Word mortgage loan Enterprise policies QuickBooks Sales sales reports telecommunications phone workflow written Martinez Adult Education, Business Training Center √Ø¬º City , State Computer Applications Specialist Certificate Program 2008 Computer Applications Company Name √Ø¬º City , State Company Name √Ø¬º City , State Company Name √Ø¬º City , State Company Name √Ø¬º City , State Company Name √Ø¬º City , State January 2011 January 2008 January 2006 January 2004 January 20

In [ ]:
from sklearn.metrics import classification_report

# Turn the predictions into integers
npredictions = [int(x) for x in predictions]

# print a classification report
print(classification_report(npredictions, sampled_df["label"]))

                                  precision    recall  f1-score   support

             Business Management       0.18      0.19      0.19        72
Civil and Mechanical Engineering       0.28      0.17      0.21       100
                  HR & Marketing       0.48      0.22      0.30        69
                   Software & IT       0.33      0.60      0.43       158
                Tech Engineering       0.46      0.21      0.29       101

                        accuracy                           0.32       500
                       macro avg       0.35      0.28      0.28       500
                    weighted avg       0.35      0.32      0.30       500



### Few Shot Prompting
Instead of using the combined text resume we used for zero shot prompting and Bert, we are using the original dataset with columns and rows so it is eaiser to generate similar prompt that aligns with the Few Shot Prompting Examples

We also extract just the numerical prediction from the model’s response (0-4), which maps to one of the job categories.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_sampled_df = resume_sampled_df.iloc[:, :-2].astype(str)
y_sampled = label_encoder.transform(resume_sampled_df['job_category'])


In [ ]:
few_shot_intro = (
    "Based on the following resume details, predict the job category from the following options:\n"
    "0: Business Management\n"
    "1: Civil and Mechanical Engineering\n"
    "2: HR & Marketing\n"
    "3: Software & IT\n"
    "4: Tech Engineering\n"
    "Answer with only the integer corresponding to the category.\n\n"

    "Example 1:\n"
    "Skills: Python, JavaScript (React), SQL, REST APIs, Git\n"
    "Degree: B.S. in Computer Science\n"
    "Positions: Full Stack Developer\n"
    "Certifications: AWS Certified Developer\n"
    "Answer: 3\n\n"

    "Example 2:\n"
    "Skills: Machine Learning, PyTorch, TensorFlow, Data Pipelines, AWS, DevOps, MLOps\n"
    "Degree: M.S. in Data Science\n"
    "Positions: Machine Learning Engineer\n"
    "Certifications: TensorFlow Developer Certificate\n"
    "Answer: 4\n\n"

    "Example 3:\n"
    "Skills: Financial Analysis, Forecasting, Budget Planning, Team Leadership\n"
    "Degree: MBA in Finance\n"
    "Positions: Business Analyst, Operations Manager\n"
    "Certifications: Certified Management Accountant (CMA)\n"
    "Answer: 0\n"

    "Example 4:\n"
    "Skills: Data Analysis, SQL, Big Data Pipelines, Airflow, AWS Redshift\n"
    "Degree: B.Tech in Data Engineering\n"
    "Positions: Data Engineer, Analytics Engineer\n"
    "Certifications: Google Cloud Professional Data Engineer\n"
    "Answer: 4\n"
)


In [ ]:
import time
import pandas as pd
import re

# Store cleaned predictions
predictions = []

# Loop through the sampled resumes
for index, row in resume_sampled_df.iterrows():
    # Combine all fields
    resume_details = (
        f"Skills: {row.get('skills', '')}\n"
        f"{row.get('degree_names', '')}, {row.get('passing_years', '')}, {row.get('major_field_of_studies', '')}\n"
        f"Positions: {row.get('positions', '')}\n"
        f"Certifications: {row.get('certification_providers', '')} - {row.get('certification_skills', '')}\n"
    )

    # Build the prompt
    prompt = few_shot_intro + "\nResume:\n" + resume_details + "\nAnswer:"

    # API message format
    MESSAGES = [{"role": "user", "content": prompt}]

    # Make API call
    completion = client.chat.complete(
        model=MODEL,
        messages=MESSAGES
    )

    # Get response
    response = completion.choices[0].message.content.strip()
    print(f"\nPrompt:\n{prompt}\nResponse:\n{response}")

    # Extract only the number (0-4) from response
    match = re.search(r"\b([0-4])\b", response)
    if match:
        predictions.append(int(match.group(1)))
    else:
        print(f"Warning: Unexpected format at row {index}")
        predictions.append("ERROR")

    time.sleep(5)


Streaming output truncated to the last 5000 lines.
Example 3:
Skills: Financial Analysis, Forecasting, Budget Planning, Team Leadership
Degree: MBA in Finance
Positions: Business Analyst, Operations Manager
Certifications: Certified Management Accountant (CMA)
Answer: 0
Example 4:
Skills: Data Analysis, SQL, Big Data Pipelines, Airflow, AWS Redshift
Degree: B.Tech in Data Engineering
Positions: Data Engineer, Analytics Engineer
Certifications: Google Cloud Professional Data Engineer
Answer: 4

Resume:
Skills: Machine Learning Engineer Data Analyst Natural Language Processing Deep Learning Reinforcement Learning Tensorflow Keras Scikit Learn Numpy Pandas Matplotlib Python
B.Tech M.Tech, 2017, Electronics/Telecommunication Advanced Analytics
Positions: Analyst Intern
Certifications: nan - nan

Answer:
Response:
4

Prompt:
Based on the following resume details, predict the job category from the following options:
0: Business Management
1: Civil and Mechanical Engineering
2: HR & Marketing

In [ ]:
from sklearn.metrics import classification_report

# Turn the predictions into integers
npredictions = [int(x) for x in predictions]

# print a classification report
print(classification_report(npredictions, sampled_df["label"]))

                                  precision    recall  f1-score   support

             Business Management       0.13      0.18      0.15        72
Civil and Mechanical Engineering       0.16      0.18      0.17       100
                  HR & Marketing       0.10      0.06      0.07        69
                   Software & IT       0.29      0.16      0.20       158
                Tech Engineering       0.22      0.35      0.27       101

                        accuracy                           0.19       500
                       macro avg       0.18      0.18      0.17       500
                    weighted avg       0.20      0.19      0.18       500

